In [8]:
import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
import japanize_matplotlib
import os

In [ ]:
size_array = [1000,2000,4000,8000,16000,32000,64000,128000,256000]
def makefigure_DPUtime():
    x_axis = []
    result = []
    result_insert = []
    result_get= []
    result_total  = []
    df = pd.read_csv("data/requestnum.csv")
    result += df['DPU実行時間'].values.tolist()
    result_insert += df['insert平均サイクル'].values.tolist()
    result_get += df['get平均サイクル'].values.tolist()
    x_axis += df['挿入するキーの数'].values.tolist()
    plt.figure()
    plt.title("requestnum_DPUtime")
    plt.xlabel('num of requests')
    plt.ylabel('Time[s]')
    plt.plot(x_axis, result, label='total', marker='o')
    plt.ylim(0,)
    plt.xticks([1000]+x_axis[5:],["1000"]+[str(num) for num in x_axis[5:]])
    plt.grid()
    plt.savefig("graphs/requestnum_DPUtime.png")
    plt.show()

def makefigure_requestnum():
    x_axis = []
    result_insert = []
    result_get= []

    df = pd.read_csv("data/test.csv")
    #result += df['Time'].values.tolist()
    result_insert += df['cycles per insert'].values.tolist()
    result_get += df['cycles per get'].values.tolist()
    x_axis += df['num_of_requests'].values.tolist()
    plt.figure()
    plt.title("requestnum_averagecycle")
    plt.xlabel('num of requests')
    plt.ylabel('average DPU cycles')
    plt.plot(x_axis, result_insert, label='insert', marker='o')
    plt.plot(x_axis, result_get, label='get', marker='o')
    plt.ylim(0,)
    #plt.xscale('log')
    plt.xticks([1000]+x_axis[6:],["1000"]+[str(num) for num in x_axis[6:]])
    #plt.yscale('log')
    plt.legend()
    plt.grid()
    plt.savefig("graphs/test.png")
    plt.show()
    
def makefigure_taskletnum():
    x_axis = []
    throughput = []
    throughput_skew = []
    df = pd.read_csv("data/taskletnum_upmem_pointerdram.csv")
    #result += df['Time'].values.tolist()
    throughput += df[' throughput[OPS/s]'].values.tolist()
    for i in range (0, len(throughput)):
        throughput[i] = throughput[i]/1000000.0
        throughput_skew.append(0.040306)
    x_axis += df['num of tasklets'].values.tolist()
    fig = plt.figure()
    plt.rcParams["font.size"] = 18
    plt.rcParams["savefig.dpi"] = 300
    plt.title("スレッド数とスループットの関係")
    plt.xlabel('各DPUのスレッド数')
    plt.ylabel('スループット[MOPS/s]')
    plt.plot(x_axis, throughput, marker='o')
    plt.plot(x_axis, throughput_skew)
    plt.xlim(0,)
    plt.ylim(0,)
    #plt.xscale('log')
    #plt.xticks([1000]+x_axis[6:],["1000"]+[str(num) for num in x_axis[6:]])
    #plt.yscale('log')
    plt.grid()
    fig.subplots_adjust(bottom=0.13,left=0.15,top=0.93,right=0.99)
    plt.savefig("graphs/num_of_tasklets_pointerdram.png",transparent = True)
    plt.show()
    
def makefigure_taskletnum_time():
    x_axis = []
    time_sendreq = []
    time_dpuexe = []
    df = pd.read_csv("data/taskletnum_upmem_pointerdram.csv")
    time_sendreq += df[' time_sendrequests[s]'].values.tolist()
    time_dpuexe += df[' time_dpu_execution[s]'].values.tolist()
    x_axis += df['num of tasklets'].values.tolist()
    fig = plt.figure()
    plt.rcParams["font.size"] = 18
    plt.rcParams["savefig.dpi"] = 300
    plt.title("実行時間の内訳")
    plt.xlabel('各DPUのスレッド数')
    plt.ylabel('時間[s]')
    plt.bar(x_axis, time_sendreq, label="クエリ送信時間(CPU→DPU)",align="center", color = "c")
    plt.bar(x_axis, time_dpuexe, label="DPU実行時間",align="center", color = "b",bottom=time_sendreq)  
    plt.ylim(0,)
    #plt.xscale('log')
    #plt.xticks([1000]+x_axis[6:],["1000"]+[str(num) for num in x_axis[6:]])
    #plt.yscale('log')
    plt.legend()
    #plt.grid()
    fig.subplots_adjust(bottom=0.13,left=0.15,top=0.93,right=0.99)
    plt.savefig("graphs/num_of_tasklets_time_pointerdram.png",transparent = True)
    plt.show()
    
def makefigure_numcpu_throughput_ratio():
    zipf_const = ["0", "0.99", "1.2"]
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax2 = ax1.twinx()
    for a in zipf_const:
        x_axis = []
        throughput = []
        cpu_percent = []
        df = pd.read_csv("data/cpu_and_dpu_zipfianconst" + a + ".csv")
        #result += df['Time'].values.tolist()
        throughput += df[' throughput'].values.tolist()
        cpu_percent += df[' num_reqs_{cpu/(cpu+dpu)}'].values.tolist()
        for i in range (0, len(throughput)):
            throughput[i] = throughput[i]/1000000.0
        x_axis += df[' num_CPU_Trees'].values.tolist()
        plt.rcParams["savefig.dpi"] = 300
        plt.title("throughput",fontsize=18)
        plt.xlabel('number of Trees in CPU',fontsize=18)
        ax1.set_ylabel('throughput[MOPS/s]',fontsize=18)
        ax2.set_ylabel('query ratio in cpu[%]',fontsize=18)
        ax1.plot(x_axis, throughput, marker='o', label="zipf_const=" + a)
        ax2.plot(x_axis, cpu_percent, marker='x', label="zipf_const=" + a)
        #plt.xlim(0,)
        #plt.ylim(0,)
        plt.xscale('log')
        #plt.xticks([1000]+x_axis[6:],["1000"]+[str(num) for num in x_axis[6:]])
        ax1.set_yscale('log')
        plt.grid()
        fig.subplots_adjust()
    plt.legend()
    plt.savefig("graphs/cpu_and_dpu.png",transparent = False)
    plt.show()



In [14]:
def makefigure_numcpu_throughput():
    zipf_const = ["0", "0.99", "1.2"]
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    for a in zipf_const:
        x_axis = []
        throughput = []
        cpu_percent = []
        df = pd.read_csv("../data/cpu_and_dpu_zipfianconst" + a + ".csv")
        #result += df['Time'].values.tolist()
        throughput += df[' throughput'].values.tolist()
        cpu_percent += df[' num_reqs_{cpu/(cpu+dpu)}'].values.tolist()
        for i in range (0, len(throughput)):
            throughput[i] = throughput[i]/1000000.0
        x_axis += df[' num_CPU_Trees'].values.tolist()
        plt.rcParams["savefig.dpi"] = 300
        #plt.title("CPU Skew",fontsize=18)
        plt.xlabel('Number of Trees in CPU',fontsize=18)
        ax1.set_ylabel('Throughput[MOPS/s]',fontsize=18)
        ax1.plot(x_axis, throughput, marker='o', label="Zipfian Const=" + a)
        #plt.xlim(0,)
        #plt.ylim(0,)
        plt.xscale('log')
        #plt.xticks([1000]+x_axis[6:],["1000"]+[str(num) for num in x_axis[6:]])
        ax1.set_yscale('log')
        plt.grid()
        fig.subplots_adjust()
    plt.legend()
    plt.savefig("../graphs/cpu_and_dpu.png",transparent = False)
    plt.show()

def makefigure_numcpu_throughput_ratio():
    zipf_const = ["0", "0.99", "1.2"]
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    for a in zipf_const:
        throughput = []
        cpu_percent = []
        df = pd.read_csv("../data/cpu_and_dpu_zipfianconst" + a + ".csv")
        #result += df['Time'].values.tolist()
        throughput += df[' throughput'].values.tolist()
        cpu_percent += df[' num_reqs_{cpu/(cpu+dpu)}'].values.tolist()
        for i in range (0, len(throughput)):
            throughput[i] = throughput[i]/1000000.0
        plt.rcParams["savefig.dpi"] = 300
        plt.title("throughput",fontsize=18)
        plt.xlabel('CPU Query Ratio',fontsize=18)
        ax1.set_ylabel('throughput[MOPS/s]',fontsize=18)
        ax1.plot(cpu_percent, throughput, marker='o', label="zipf_const=" + a)
        #plt.xlim(0,)
        #plt.ylim(0,)
        #plt.xscale('log')
        #plt.xticks([1000]+x_axis[6:],["1000"]+[str(num) for num in x_axis[6:]])
        ax1.set_yscale('log')
        plt.grid()
        fig.subplots_adjust()
    plt.legend()
    plt.savefig("../graphs/cpu_and_dpu_cpuratio.png",transparent = False)
    plt.show()

In [79]:
def makefigure_cyclic():
    zipf_const = ["0", "0.6", "0.99", "1.2"]
    cyclic = ["cyclic_",""]
    work_load = ["w05r95", "w50r50"]
    x = [4,16,64,256,512,1024,1536,2047]
    y = [16,32,64,256,512,1024]
    X,Y = np.meshgrid(x,y)
    plt.rcParams["savefig.dpi"] = 300
    for a in zipf_const:
        for b in cyclic:
            for c in work_load:
                throughput = []
                cpu_percent = []
                df = pd.read_csv("../data/" + b + "redundant_num_bptree_in_cpu_alpha" + a + "_" + c + ".csv")
                fig, ax = plt.subplots()
                #result += df['Time'].values.tolist()
                throughput += df[' throughput'].values.tolist()[:48]
                for i in range (0, len(throughput)):
                    throughput[i] = throughput[i]/1000000.0
                Z = np.reshape(throughput,[6,8])
                cpu_percent += df[' exec_time_{cpu/(cpu&dpu)}[%]'].values.tolist()
                contf = ax.contourf(X, Y, Z, 20, cmap='Blues')
                #ax.set_aspect('equal','box')
                plt.colorbar(contf)

                plt.title(b + a + "_" + c)
                plt.xlabel('Num DPUs singleBPTree',fontsize=18)
                ax.set_ylabel('Num BPTrees in CPU',fontsize=18)
                #ax.plot(x_axis, throughput, marker='o', label="Zipfian Const=" + a)
                #plt.xlim(0,)
                #plt.ylim(0,)

                ax.set_xticks(x)
                ax.set_yticks(y)
                ax.set_xscale('log')
                ax.set_yscale('log')
                #plt.grid()
                #plt.legend()
                fig.subplots_adjust()
                plt.savefig("../graphs/" + b + "redundant_alpha" + a + "_" + c + ".png",transparent = False)
                plt.show()

def show_max_throughputs():
    zipf_const = ["0", "0.6", "0.99", "1.2"]
    cyclic = ["cyclic_",""]
    work_load = ["w05r95", "w50r50"]
    x = [4,16,64,256,512,1024,1536,2047]
    y = [16,32,64,256,512,1024]
    X,Y = np.meshgrid(x,y)
    plt.rcParams["savefig.dpi"] = 300
    for a in zipf_const:
        for b in cyclic:
            for c in work_load:
                throughput = []
                num_dpus_redundant = []
                num_CPU_Trees = []
                df = pd.read_csv("../data/" + b + "redundant_num_bptree_in_cpu_alpha" + a + "_" + c + ".csv")
                throughput += df[' throughput'].values.tolist()[:48]
                num_dpus_redundant += df[' num_dpus_redundant'].values.tolist()[:48]
                num_CPU_Trees += df[' num_CPU_Trees'].values.tolist()[:48]
                max_throughput = 0
                max_i = 0
                for i in range (0, len(throughput)):
                    throughput[i] = throughput[i]/1000000.0
                    if(max_throughput < throughput[i]):
                        max_throughput = throughput[i]
                        max_i = i
                print("alpha=" +a + ", cyclic=" + b + ", work_load=" + c + ": max_throughput=" + str(max_throughput) + ", num_dpus_redundant=" + str(num_dpus_redundant[max_i])+ ", num_CPU_Trees="  + str(num_CPU_Trees[max_i]))



In [80]:
show_max_throughputs()

alpha=0, cyclic=cyclic_, work_load=w05r95: max_throughput=625.74304, num_dpus_redundant=16, num_CPU_Trees=256
alpha=0, cyclic=cyclic_, work_load=w50r50: max_throughput=351.276512, num_dpus_redundant=1, num_CPU_Trees=256
alpha=0, cyclic=, work_load=w05r95: max_throughput=621.832576, num_dpus_redundant=16, num_CPU_Trees=64
alpha=0, cyclic=, work_load=w50r50: max_throughput=351.464192, num_dpus_redundant=1, num_CPU_Trees=256
alpha=0.6, cyclic=cyclic_, work_load=w05r95: max_throughput=246.869712, num_dpus_redundant=1024, num_CPU_Trees=256
alpha=0.6, cyclic=cyclic_, work_load=w50r50: max_throughput=205.359456, num_dpus_redundant=1024, num_CPU_Trees=32
alpha=0.6, cyclic=, work_load=w05r95: max_throughput=255.674048, num_dpus_redundant=1024, num_CPU_Trees=512
alpha=0.6, cyclic=, work_load=w50r50: max_throughput=115.774376, num_dpus_redundant=1536, num_CPU_Trees=16
alpha=0.99, cyclic=cyclic_, work_load=w05r95: max_throughput=211.774224, num_dpus_redundant=1024, num_CPU_Trees=16
alpha=0.99, cyc

In [9]:
os.getcwd()

'/home/okudahi/upmem-sdk/bp-forest-exp/scripts'